## Libraries

In [1]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a372dcffd16c5dd200aca84451cfe0d8a5fe102f48c0f7b91150082e60e3296d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, AutoTokenizer, TrainerCallback
from rouge_score import rouge_scorer
from datasets import Dataset
import pandas as pd
import torch
import json
import os

2025-05-07 11:11:54.361623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746616314.560761      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746616314.612161      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Dataset

In [3]:
folder_path = '/kaggle/input/amazon-products-dataset/'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
csv_files

['Gaming Consoles.csv',
 'Car Electronics.csv',
 'Janitorial and Sanitation Supplies.csv',
 'All Electronics.csv',
 'All Books.csv',
 'Make-up.csv',
 'Travel Accessories.csv',
 'Indian Language Books.csv',
 'Car and Bike Care.csv',
 'Sunglasses.csv',
 'Bags and Luggage.csv',
 'Yoga.csv',
 'Sportswear.csv',
 'Fiction Books.csv',
 'Exam Central.csv',
 'Home Storage.csv',
 'Toys Gifting Store.csv',
 'All English.csv',
 'Amazon-Products.csv',
 'Air Conditioners.csv',
 'Shoes.csv',
 'Casual Shoes.csv',
 'Baby Products.csv',
 'Sports Collectibles.csv',
 'Wallets.csv',
 'Musical Instruments and Professional Audio.csv',
 'Gold and Diamond Jewellery.csv',
 'Nursing and Feeding.csv',
 'Home Furnishing.csv',
 'School Textbooks.csv',
 'All Hindi.csv',
 'Baby Bath Skin and Grooming.csv',
 'Coffee Tea and Beverages.csv',
 'Headphones.csv',
 'Furniture.csv',
 'Shirts.csv',
 'Subscribe and Save.csv',
 'Fitness Accessories.csv',
 'Formal Shoes.csv',
 'Cycling.csv',
 'Western Wear.csv',
 'Bedroom Linen.

In [4]:
dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Electronic Spices 2.75 Inch 4ω (Ohm) 400w Max ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61G5k3T0ff...,https://www.amazon.in/Electronic-Spices-Power-...,NaN,NaN,₹129,₹199,NaN
1,"ZQWINT Bluetooth Car Adapter, Mini USB Bluetoo...",car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/51NLOKwNsL...,https://www.amazon.in/ZQWINT-Bluetooth-Transmi...,NaN,NaN,₹219,₹999,NaN
2,PROTECTRON 6.35X32mm GLASS FUSE/INVERTER FUSE ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/71St2ruv+N...,https://www.amazon.in/PROTECTRON-6-35X32mm-GLA...,3.8,32,NaN,₹105,NaN
3,Cave Maruti Suzuki Male-Female Stereo Coupler ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/31npb8UF2y...,https://www.amazon.in/Maruti-Suzuki-Stereo-Cou...,4.1,7,₹582,₹873,NaN
4,COVERBLACK Rubber Back Cover for Infinix X6815...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/611zXbVxbN...,https://www.amazon.in/COVERBLACK-Infinix-X6815...,NaN,NaN,₹148,₹799,NaN


In [5]:
df.shape

(1103170, 10)

In [6]:
df.columns

Index(['name', 'main_category', 'sub_category', 'image', 'link', 'ratings',
       'no_of_ratings', 'discount_price', 'actual_price', 'Unnamed: 0'],
      dtype='object')

In [7]:
df['main_category'].unique()

array(['car & motorbike', 'industrial supplies', 'tv, audio & cameras',
       'beauty & health', 'bags & luggage', 'accessories',
       'sports & fitness', 'stores', 'home & kitchen',
       'toys & baby products', 'appliances', 'grocery & gourmet foods',
       'pet supplies', "kids' fashion", "women's shoes", "men's shoes",
       "women's clothing", "men's clothing", 'music',
       'home, kitchen, pets'], dtype=object)

In [8]:
df['sub_category'].unique()

array(['Car Electronics', 'Janitorial & Sanitation Supplies',
       'All Electronics', 'Make-up', 'Travel Accessories',
       'Car & Bike Care', 'Sunglasses', 'Bags & Luggage', 'Yoga',
       'Sportswear', 'Home Storage', 'Toys Gifting Store',
       'Air Conditioners', 'All Appliances',
       'All Car & Motorbike Products', 'All Exercise & Fitness',
       'All Grocery & Gourmet Foods', 'All Home & Kitchen',
       'All Pet Supplies', 'All Sports, Fitness & Outdoors',
       'Amazon Fashion', 'Baby Bath, Skin & Grooming', 'Baby Fashion',
       'Baby Products', 'Backpacks', 'Badminton', 'Ballerinas',
       'Beauty & Grooming', 'Bedroom Linen', 'Camera Accessories',
       'Cameras', 'Camping & Hiking', 'Car Accessories', 'Car Parts',
       'Cardio Equipment', 'Casual Shoes', 'Clothing',
       'Coffee, Tea & Beverages', 'Cricket', 'Cycling', 'Diapers',
       'Diet & Nutrition', 'Dog supplies', 'Ethnic Wear',
       'Fashion & Silver Jewellery', 'Fashion Sales & Deals',
       'F

In [9]:
df.isnull().sum()

name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings           351588
no_of_ratings     351588
discount_price    122326
actual_price       35626
Unnamed: 0        551585
dtype: int64

In [10]:
df.duplicated().sum()

0

In [11]:
# remove ₹ char 
def clean_price(value):
    if isinstance(value, str):
        # Remove currency symbols and commas
        return value.replace('₹', '').replace(',', '')
    return value

df['discount_price'] = df['discount_price'].apply(clean_price)
df['actual_price'] = df['actual_price'].apply(clean_price)

# convert data type to numerical
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce') 
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')
df['discount_price'] = pd.to_numeric(df['discount_price'], errors='coerce')
df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')

# fill missing numerical values with the median
df['ratings'] = df['ratings'].fillna(df['ratings'].median())
df['no_of_ratings'] = df['no_of_ratings'].fillna(df['no_of_ratings'].median())
df['discount_price'] = df['discount_price'].fillna(df['discount_price'].median())
df['actual_price'] = df['actual_price'].fillna(df['actual_price'].median())

# drop the unnecessary column 'Unnamed: 0'
df = df.drop(columns=['Unnamed: 0'])

df.isnull().sum()

name              0
main_category     0
sub_category      0
image             0
link              0
ratings           0
no_of_ratings     0
discount_price    0
actual_price      0
dtype: int64

In [12]:
df.shape

(1103170, 9)

In [13]:
df.duplicated().sum()

551585

In [14]:
df = pd.read_csv('/kaggle/input/amazon-products-dataset/Home Dcor.csv')
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,RAG28 Wooden Wall Decor Set of 11 (SPW1) Multi...,home & kitchen,Home Décor,https://m.media-amazon.com/images/I/81GQfgl4cn...,https://www.amazon.in/RAG28-Wooden-Multicolour...,3.9,"2,657",₹284.05,₹799
1,"ArtX Paper Motivational Wall Frames, Inspirati...",home & kitchen,Home Décor,https://m.media-amazon.com/images/I/61VDPBbPgH...,https://www.amazon.in/ArtX-Motivational-Quotes...,4.4,572,₹593,"₹1,399"
2,Mahogany Life Vintage Glass Hanging Photo Fram...,home & kitchen,Home Décor,https://m.media-amazon.com/images/I/81dRciG8gB...,https://www.amazon.in/Mahogany-Life-Vintage-Ha...,4.2,210,₹469,₹901
3,"Indianara Religious Painting -Synthetic Wood, ...",home & kitchen,Home Décor,https://m.media-amazon.com/images/I/91aG2syqs9...,https://www.amazon.in/Indianara-Panchmukhi-Syn...,4.5,589,₹182,₹499
4,Lexton 20 Piece Photo Clips String Light Batte...,home & kitchen,Home Décor,https://m.media-amazon.com/images/I/51emgfV2Mf...,https://www.amazon.in/Lexton-Included-Christma...,4.0,"1,539",₹244.99,₹499


In [15]:
df.shape

(1272, 9)

In [16]:
df.isnull().sum()

name               0
main_category      0
sub_category       0
image              0
link               0
ratings           82
no_of_ratings     82
discount_price    50
actual_price       4
dtype: int64

In [17]:
df = df.dropna()
df.isnull().sum()

name              0
main_category     0
sub_category      0
image             0
link              0
ratings           0
no_of_ratings     0
discount_price    0
actual_price      0
dtype: int64

In [18]:
df.duplicated().sum()

0

In [19]:
df['input_text'] =  f"Product name: {df['name']}, Category: {df['main_category']}, Sub-category: {df['sub_category']}, Rating: {df['ratings'].fillna('0').astype(str)}, Price: {df['actual_price'].fillna('0').astype(str)}"

df['target_text'] = "This is a great product for " + df['main_category'] + " users."

df[['input_text', 'target_text']].head()

,input_text,target_text
0,Product name: 0 RAG28 Wooden Wall Decor ...,This is a great product for home & kitchen users.
1,Product name: 0 RAG28 Wooden Wall Decor ...,This is a great product for home & kitchen users.
2,Product name: 0 RAG28 Wooden Wall Decor ...,This is a great product for home & kitchen users.
3,Product name: 0 RAG28 Wooden Wall Decor ...,This is a great product for home & kitchen users.
4,Product name: 0 RAG28 Wooden Wall Decor ...,This is a great product for home & kitchen users.


## Tokenization

In [20]:
# load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [21]:
def tokenize_sample(row, idx):
    if (idx + 1) % 100 == 0:
        print(f"sample {idx + 1}")
    input_enc = tokenizer(
        row['input_text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

    target_enc = tokenizer(
        row['target_text'],
        padding='max_length',
        truncation=True,
        max_length=512
    )

    input_enc['labels'] = target_enc['input_ids']
    return input_enc


train_dataset = df[['input_text', 'target_text']].apply(lambda x: tokenize_sample(x, x.name), axis=1).tolist()

train_dataset = Dataset.from_list(train_dataset)

sample 100
sample 200
sample 300
sample 400
sample 500
sample 700
sample 800
sample 900
sample 1000
sample 1100
sample 1200


In [57]:
train_dataset[0]

{'input_ids': [15667,
  1438,
  25,
  657,
  220,
  220,
  220,
  220,
  220,
  220,
  371,
  4760,
  2078,
  40143,
  5007,
  4280,
  273,
  5345,
  286,
  1367,
  357,
  4303,
  54,
  16,
  8,
  15237,
  986,
  198,
  16,
  220,
  220,
  220,
  220,
  220,
  220,
  3683,
  55,
  14962,
  6543,
  452,
  864,
  5007,
  36291,
  11,
  25300,
  343,
  7246,
  986,
  198,
  17,
  220,
  220,
  220,
  220,
  220,
  220,
  8882,
  519,
  1092,
  5155,
  34057,
  12158,
  367,
  4924,
  5555,
  15183,
  986,
  198,
  18,
  220,
  220,
  220,
  220,
  220,
  220,
  3942,
  3301,
  23945,
  48892,
  532,
  13940,
  429,
  6587,
  5326,
  11,
  2644,
  198,
  19,
  220,
  220,
  220,
  220,
  220,
  220,
  1004,
  22874,
  1160,
  27053,
  5555,
  1012,
  2419,
  10903,
  4401,
  6577,
  660,
  986,
  198,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  220,
  

## setting up the model

In [22]:
# load the pre-trained model (GPT-2)
model = GPT2LMHeadModel.from_pretrained('gpt2')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [48]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=100,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [49]:
#device = torch.device('cpu')
#model = model.to(device)

In [50]:
class LoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 10 == 0:
            print(f"Step {state.global_step} / {state.max_steps}")

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"Epoch {state.epoch:.0f} ended. Total steps: {state.global_step}")

    def on_epoch_start(self, args, state, control, **kwargs):
        print(f"Epoch {state.epoch:.0f} start")

# trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    callbacks=[LoggingCallback()]
)

In [51]:
os.environ["WANDB_DISABLED"] = "true"

In [52]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [54]:
# start training
trainer.train()

Step,Training Loss
500,0.003600


Step 10 / 858
Step 20 / 858
Step 30 / 858
Step 40 / 858
Step 50 / 858
Step 60 / 858
Step 70 / 858
Step 80 / 858
Step 90 / 858
Step 100 / 858
Step 110 / 858
Step 120 / 858
Step 130 / 858
Step 140 / 858
Step 150 / 858
Step 160 / 858
Step 170 / 858
Step 180 / 858
Step 190 / 858
Step 200 / 858
Step 210 / 858
Step 220 / 858
Step 230 / 858
Step 240 / 858
Step 250 / 858
Step 260 / 858
Step 270 / 858
Step 280 / 858
Epoch 1 ended. Total steps: 286
Step 290 / 858
Step 300 / 858
Step 310 / 858
Step 320 / 858
Step 330 / 858
Step 340 / 858
Step 350 / 858
Step 360 / 858
Step 370 / 858
Step 380 / 858
Step 390 / 858
Step 400 / 858
Step 410 / 858
Step 420 / 858
Step 430 / 858
Step 440 / 858
Step 450 / 858
Step 460 / 858
Step 470 / 858
Step 480 / 858
Step 490 / 858
Step 500 / 858
Step 510 / 858
Step 520 / 858
Step 530 / 858
Step 540 / 858
Step 550 / 858
Step 560 / 858
Step 570 / 858
Epoch 2 ended. Total steps: 572
Step 580 / 858
Step 590 / 858
Step 600 / 858
Step 610 / 858
Step 620 / 858
Step 630 / 858


TrainOutput(global_step=858, training_loss=0.0026646274508851948, metrics={'train_runtime': 356.8285, 'train_samples_per_second': 9.61, 'train_steps_per_second': 2.405, 'total_flos': 914783404032000.0, 'train_loss': 0.0026646274508851948, 'epoch': 3.0})

## Generate product description

In [60]:
train_dataset = pd.DataFrame(train_dataset)
train_dataset.head()

,input_ids,attention_mask,labels
0,"[15667, 1438, 25, 657, 220, 220, 220, 220, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1212, 318, 257, 1049, 1720, 329, 1363, 1222, ..."
1,"[15667, 1438, 25, 657, 220, 220, 220, 220, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1212, 318, 257, 1049, 1720, 329, 1363, 1222, ..."
2,"[15667, 1438, 25, 657, 220, 220, 220, 220, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1212, 318, 257, 1049, 1720, 329, 1363, 1222, ..."
3,"[15667, 1438, 25, 657, 220, 220, 220, 220, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1212, 318, 257, 1049, 1720, 329, 1363, 1222, ..."
4,"[15667, 1438, 25, 657, 220, 220, 220, 220, 220...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1212, 318, 257, 1049, 1720, 329, 1363, 1222, ..."


In [62]:
# generate description based on the product info
def generate_description(row):
    input_ids = torch.tensor(row['input_ids']).unsqueeze(0)
    
    # Generate the description
    output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return generated_text

train_dataset['generated_description'] = train_dataset.apply(generate_description, axis=1)

train_dataset[['name', 'generated_description']].head()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ValueError: Input length of input_ids is 512, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

## Evaluation

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score("Actual product description here.", generated_text)

scores